# NaiveBayes Classifier

1. 주어진 데이터를 전처리한다.
2. NaiveBayes 분류기 모델을 구현하고 학습 데이터로 이를 학습시킨다.
3. Test case 로 결과를 확인한다.

## 라이브러리

In [1]:
# konlpy 설치
!pip3 install konlpy

     |████████████████████████████████| 19.4 MB 510 kB/s eta 0:00:01
     |████████████████████████████████| 86 kB 833 kB/s eta 0:00:01
     |████████████████████████████████| 377 kB 925 kB/s eta 0:00:01


In [2]:
from tqdm import tqdm
# 다양한 한국어 형태소 분석기가 클래스로 구현되어 있음
from konlpy import tag
from collections import defaultdict
import math
import warnings
warnings.filterwarnings('ignore')

## 학습 및 테스트 데이터 전처리

Sample 데이터를 확인한다.
긍정(1), 부정(0) 2가지 class 로 되어 있다.

In [3]:
train_data = [
  "정말 맛있습니다. 추천합니다.",
  "기대했던 것보단 별로였네요.",
  "다 좋은데 가격이 너무 비싸서 다시 가고 싶다는 생각이 안 드네요.",
  "완전 최고입니다! 재방문 의사 있습니다.",
  "음식도 서비스도 다 만족스러웠습니다.",
  "위생 상태가 좀 별로였습니다. 좀 더 개선되기를 바랍니다.",
  "맛도 좋았고 직원분들 서비스도 너무 친절했습니다.",
  "기념일에 방문했는데 음식도 분위기도 서비스도 다 좋았습니다.",
  "전반적으로 음식이 너무 짰습니다. 저는 별로였네요.",
  "위생에 조금 더 신경 썼으면 좋겠습니다. 조금 불쾌했습니다."
]
train_labels = [1, 0, 0, 1, 1, 0, 1, 1, 0, 0]

test_data = [
  "정말 좋았습니다. 또 가고 싶네요.",
  "별로였습니다. 되도록 가지 마세요.",
  "다른 분들께도 추천드릴 수 있을 만큼 만족했습니다.",
  "서비스가 좀 더 개선되었으면 좋겠습니다. 기분이 좀 나빴습니다."
]

KoNLPy 패키지에서 제공하는 Twitter(Okt) tokenizer(클래스) 를 사용하여 tokenization 한다.  
tokenizer class 에 따라 결과가 다르다.

In [8]:
tokenizer = tag.Okt() # Twitter

In [9]:
def make_tokenized(data):
    tokenized = [] # 단어 단위로 나뉜 리뷰 데이터
    
    for sent in tqdm(data):
        tokens = tokenizer.morphs(sent)
        tokenized.append(tokens)
        
    return tokenized

In [10]:
train_tokenized = make_tokenized(train_data)
test_tokenized = make_tokenized(test_data)

100%|██████████| 4/4 [00:00<00:00, 224.07it/s]


In [12]:
train_tokenized

[['정말', '맛있습니다', '.', '추천', '합니다', '.'],
 ['기대했던', '것', '보단', '별로', '였네요', '.'],
 ['다',
  '좋은데',
  '가격',
  '이',
  '너무',
  '비싸서',
  '다시',
  '가고',
  '싶다는',
  '생각',
  '이',
  '안',
  '드네',
  '요',
  '.'],
 ['완전', '최고', '입니다', '!', '재', '방문', '의사', '있습니다', '.'],
 ['음식', '도', '서비스', '도', '다', '만족스러웠습니다', '.'],
 ['위생',
  '상태',
  '가',
  '좀',
  '별로',
  '였습니다',
  '.',
  '좀',
  '더',
  '개선',
  '되',
  '기를',
  '바랍니다',
  '.'],
 ['맛', '도', '좋았고', '직원', '분들', '서비스', '도', '너무', '친절했습니다', '.'],
 ['기념일',
  '에',
  '방문',
  '했는데',
  '음식',
  '도',
  '분위기',
  '도',
  '서비스',
  '도',
  '다',
  '좋았습니다',
  '.'],
 ['전반', '적', '으로', '음식', '이', '너무', '짰습니다', '.', '저', '는', '별로', '였네요', '.'],
 ['위생', '에', '조금', '더', '신경', '썼으면', '좋겠습니다', '.', '조금', '불쾌했습니다', '.']]

In [13]:
test_tokenized

[['정말', '좋았습니다', '.', '또', '가고', '싶네요', '.'],
 ['별로', '였습니다', '.', '되도록', '가지', '마세요', '.'],
 ['다른', '분', '들께도', '추천', '드릴', '수', '있을', '만큼', '만족했습니다', '.'],
 ['서비스',
  '가',
  '좀',
  '더',
  '개선',
  '되었으면',
  '좋겠습니다',
  '.',
  '기분',
  '이',
  '좀',
  '나빴습니다',
  '.']]

학습데이터 기준으로 가장 많이 등장한 단어부터 순서대로 vocab 에 추가합니다.

In [15]:
word_count = defaultdict(int) # key : 단어, value : 등장횟수

for tokens in tqdm(train_tokenized): # 각 단어에 대해
    for token in tokens:
        word_count[token] += 1

100%|██████████| 10/10 [00:00<00:00, 94254.02it/s]


In [16]:
word_count # 각 단어에 대한 빈도

defaultdict(int,
            {'정말': 1,
             '맛있습니다': 1,
             '.': 14,
             '추천': 1,
             '합니다': 1,
             '기대했던': 1,
             '것': 1,
             '보단': 1,
             '별로': 3,
             '였네요': 2,
             '다': 3,
             '좋은데': 1,
             '가격': 1,
             '이': 3,
             '너무': 3,
             '비싸서': 1,
             '다시': 1,
             '가고': 1,
             '싶다는': 1,
             '생각': 1,
             '안': 1,
             '드네': 1,
             '요': 1,
             '완전': 1,
             '최고': 1,
             '입니다': 1,
             '!': 1,
             '재': 1,
             '방문': 2,
             '의사': 1,
             '있습니다': 1,
             '음식': 3,
             '도': 7,
             '서비스': 3,
             '만족스러웠습니다': 1,
             '위생': 2,
             '상태': 1,
             '가': 1,
             '좀': 2,
             '였습니다': 1,
             '더': 2,
             '개선': 1,
             '되': 1,
             '기를': 1,
     

In [20]:
word_count = sorted(word_count.items(), key = lambda x: x[1], reverse=True) # 빈도순 처리
print(len(word_count))

66


In [24]:
w2i = {} # key : 단어, value : 단어의 index
for pair in tqdm(word_count):
    if pair[0] not in w2i:
        w2i[pair[0]] = len(w2i) # 계속 이어서 새롭게 할당하기 for index 처리

100%|██████████| 66/66 [00:00<00:00, 544929.26it/s]


In [25]:
len(w2i)

66

In [26]:
w2i
# word_count 가 빈도순으로 처리되어있으므로, 이 index 또한 자연스럽게 빈도역순으로 인덱싱됨.

{'.': 0,
 '도': 1,
 '별로': 2,
 '다': 3,
 '이': 4,
 '너무': 5,
 '음식': 6,
 '서비스': 7,
 '였네요': 8,
 '방문': 9,
 '위생': 10,
 '좀': 11,
 '더': 12,
 '에': 13,
 '조금': 14,
 '정말': 15,
 '맛있습니다': 16,
 '추천': 17,
 '합니다': 18,
 '기대했던': 19,
 '것': 20,
 '보단': 21,
 '좋은데': 22,
 '가격': 23,
 '비싸서': 24,
 '다시': 25,
 '가고': 26,
 '싶다는': 27,
 '생각': 28,
 '안': 29,
 '드네': 30,
 '요': 31,
 '완전': 32,
 '최고': 33,
 '입니다': 34,
 '!': 35,
 '재': 36,
 '의사': 37,
 '있습니다': 38,
 '만족스러웠습니다': 39,
 '상태': 40,
 '가': 41,
 '였습니다': 42,
 '개선': 43,
 '되': 44,
 '기를': 45,
 '바랍니다': 46,
 '맛': 47,
 '좋았고': 48,
 '직원': 49,
 '분들': 50,
 '친절했습니다': 51,
 '기념일': 52,
 '했는데': 53,
 '분위기': 54,
 '좋았습니다': 55,
 '전반': 56,
 '적': 57,
 '으로': 58,
 '짰습니다': 59,
 '저': 60,
 '는': 61,
 '신경': 62,
 '썼으면': 63,
 '좋겠습니다': 64,
 '불쾌했습니다': 65}

## 모델 Class 구현

NaiveBayes Classifier 모델 클래스를 구현합니다.
* `self.k` : Smoothing 을 위한 상수
* `self.w2i` : 사전에 구한 vocab (사전)
* `self.priors` : 각 class 의 prior 확률
* `self.likelihoods` : 각 token 의 특정 class 조건 내에서의 likelihood

* prior 확률  
`prior probability` : 사전 확률 : '결과 B가 아직 관측되지 않은 단계에서 원인이 A 라는 확신의 정도' 를 확률로 나타낸 것(주관확률)   
$$P(원인A)$$   

  
* likelihood 우도  
'원인이 A 일 때 결과로서 B라는 데이터를 관측할 확신의 정도' 를 나타내는 주관확률  
다만 이미 결과는 나와있으므로 확률이 아니라 결과 B 의 원인이 A 라고 생각하는 것은 그럴만 하다는 뜻에서 우도 라는 단어를 쓴다.

$$P(결과B|원인A)$$

* Laplace smoothing  
나이브 베이즈 알고리즘을 사용할 때, 없는 단어가 나올 경우 무조건 확률은 0 가 된다.    
smoothing : 새로운 단어가 나오더라도 해당 빈도에 +1 을 해줌으로써 확률이 0 이 되는 것을 막는다.    
기존   
$$\hat{p} (x|c) = \frac {count(x,c)}{\sum_{x\in V}count(x,c)} $$
변환(빈도에 1을 더해주는 것)
$$\hat{p} (x|c) = \frac {count(x,c)+1}{\sum_{x\in V}(count(x,c)+1)} =  \frac {count(x,c)+1}{\sum_{x\in V}(count(x,c)) + \left| V \right|}$$

* Log  
추가로, 계속된 단어의 확률의 곱은 값이 매우 작아질 수 있는데, Log 를 이용해서 언더플로우를 방지 할 수 있다.    
    
    
* 나이브 베이즈 분류기  
분류 기준 : 다음에 주어진 계산의 값이 최대가 되는 클래스를 선택하는 것이다.  
$$P(C_i)\prod_{k}P(X_{j}^{k}=a_{k}|C_i)$$  
로그화  
$$ \log{P(C_i) + \log(\prod_{k}P(X_{j}^{k}=a_{k}|C_i)})$$



In [27]:
class NaiveBayesClassifier():
    def __init__(self, w2i, k = 0.1):
        self.k = k
        self.w2i = w2i
        self.priors = {} # set_priors()
        self.likelihoods = {} # set_likelihoods()
        
    # 학습
    def train(self, train_tokenized, train_labels): 
        self.set_priors(train_labels) # priors 계산
        self.set_likelihoods(train_tokenized, train_labels) # Likelihoods 계산
        
    # 추론    
    def inference(self, tokens):
        log_prob0 = 0.0
        log_prob1 = 0.0
        
        for token in tokens:
            if token in self.likelihoods: # 학습 당시 추가했던 단어에 대해서만 고려
                log_prob0 += math.log(self.likelihoods[token][0])
                log_prob1 += math.log(self.likelihoods[token][1])
                
        # 마지막에 prior 를 고려
        log_prob0 += math.log(self.priors[0])
        log_prob1 += math.log(self.priors[1])
        
        if log_prob0 >= log_prob1:
            return 0
        else:
            return 1
    
    # 각 class 의 확률
    def set_priors(self, train_labels): # train data 의 label 값
        class_counts = defaultdict(int)
        for label in tqdm(train_labels):
            class_counts[label] += 1
        
        for label, count in class_counts.items():
            self.priors[label] = class_counts[label] / len(train_labels) # 빈도를 확률로 바꾸고 클래스에 할당
    
    # 가능도
    def set_likelihoods(self, train_tokenized, train_labels):
        token_dists = {} # 각 단어의 특정 class 조건 하에서만 등장 횟수
        class_counts = defaultdict(int) # 특정 class 에서 등장한 모든 단어의 등장 횟수
        # (특정 클래스에서) -> 한 단어의 등장 횟수 / 모든 단어의 등장 횟수
        
        for i, label in enumerate(tqdm(train_labels)):
            count = 0
            for token in train_tokenized[i]:
                if token in self.w2i: # 학습 데이터로 구축한 vocab 에 있는 token 만 고려
                    if token not in token_dists:
                        token_dists[token] = {0:0, 1:0}
                    token_dists[token][label] += 1 # 각 token 의 class 별 빈도 추가
                    count += 1
            class_counts[label] += count
        # 각 등장 횟수를 따로 계산
        
        for token, dist in tqdm(token_dists.items()):
            if token not in self.likelihoods: # likelihoods 에 할당
                # 위의 변환 Laplace smoothing
                self.likelihoods[token] = {
                    0:(token_dists[token][0] + self.k) / (class_counts[0] + len(self.w2i)*self.k),
                    1:(token_dists[token][1] + self.k) / (class_counts[1] + len(self.w2i)*self.k),
                }

## 모델학습

In [28]:
classifier = NaiveBayesClassifier(w2i)
classifier.train(train_tokenized, train_labels)

100%|██████████| 66/66 [00:00<00:00, 238435.89it/s]


In [36]:
classifier.priors # 각 클래스의 확률

{1: 0.5, 0: 0.5}

In [37]:
classifier.likelihoods

{'정말': {0: 0.0015243902439024393, 1: 0.02131782945736434},
 '맛있습니다': {0: 0.0015243902439024393, 1: 0.02131782945736434},
 '.': {0: 0.12347560975609756, 1: 0.11821705426356588},
 '추천': {0: 0.0015243902439024393, 1: 0.02131782945736434},
 '합니다': {0: 0.0015243902439024393, 1: 0.02131782945736434},
 '기대했던': {0: 0.016768292682926834, 1: 0.001937984496124031},
 '것': {0: 0.016768292682926834, 1: 0.001937984496124031},
 '보단': {0: 0.016768292682926834, 1: 0.001937984496124031},
 '별로': {0: 0.04725609756097562, 1: 0.001937984496124031},
 '였네요': {0: 0.03201219512195122, 1: 0.001937984496124031},
 '다': {0: 0.016768292682926834, 1: 0.040697674418604654},
 '좋은데': {0: 0.016768292682926834, 1: 0.001937984496124031},
 '가격': {0: 0.016768292682926834, 1: 0.001937984496124031},
 '이': {0: 0.04725609756097562, 1: 0.001937984496124031},
 '너무': {0: 0.03201219512195122, 1: 0.02131782945736434},
 '비싸서': {0: 0.016768292682926834, 1: 0.001937984496124031},
 '다시': {0: 0.016768292682926834, 1: 0.001937984496124031},

## 테스트


In [30]:
preds = []
for test_tokens in tqdm(test_tokenized):
    pred = classifier.inference(test_tokens)
    preds.append(pred)

100%|██████████| 4/4 [00:00<00:00, 27594.11it/s]


In [31]:
preds

[1, 0, 1, 0]

In [35]:
test_data

['정말 좋았습니다. 또 가고 싶네요.',
 '별로였습니다. 되도록 가지 마세요.',
 '다른 분들께도 추천드릴 수 있을 만큼 만족했습니다.',
 '서비스가 좀 더 개선되었으면 좋겠습니다. 기분이 좀 나빴습니다.']